In [13]:
# Dependencies and Setup
import matplotlib.pyplot as plt
import pandas as pd
import numpy as np
import requests
import time
import json
from scipy.stats import linregress

# Import API key
from config import weather_api_key

# Incorporated citipy to determine city based on latitude and longitude
from citipy import citipy

# Output File (CSV)
output_data_file = "output_data/cities.csv"

# Range of latitudes and longitudes
lat_range = (-90, 90)
lng_range = (-180, 180)

## Generate Cities List

In [33]:
# List for holding lat_lngs and cities
coordinates = []
cities = []

# Create a set of random lat and lng combinations
lats = np.random.uniform(lat_range[0], lat_range[1], size=10) # Change size to 1500 after testing
lngs = np.random.uniform(lng_range[0], lng_range[1], size=10) # Change size to 1500 after testing
coordinates = zip(lats, lngs)

# Identify nearest city for each lat, lng combination
for coord in coordinates:
    city = citipy.nearest_city(coord[0], coord[1]).city_name
    
    # If the city is unique, then add it to a our cities list
    if city not in cities:
        cities.append(city)

# Print the city count to confirm sufficient count
len(cities)

10

In [34]:
cities

['busselton',
 'hithadhoo',
 'katsuura',
 'jamestown',
 'ushuaia',
 'constitucion',
 'labutta',
 'punta arenas',
 'sibolga',
 'illoqqortoormiut']

In [49]:
# Request weather data (temperature, humidity, cloudiness, windspeed) from OpenWeather

successful_cities = []
temp = []
humidity = []
cloudiness = []
windspeed = []

print("-----------------------------------")
print("Retrieving Data – Please be patient")
print("-----------------------------------")

for city in cities:
    
    # Avoid exceeding 60 calls per minute from API request
    time.sleep(1)
    
    # Request data from API
    city_url = f"http://api.openweathermap.org/data/2.5/weather?q={city}&appid={weather_api_key}&units=imperial"
    response = requests.get(city_url).json()
    
    try:
        temp.append(response["main"]["temp"])
        humidity.append(response["main"]["humidity"])
        cloudiness.append(response["clouds"]["all"])
        windspeed.append(response["wind"]["speed"])
        successful_cities.append(city)
        print(f"Found data for city #{len(successful_cities)}: {city}")
        
    except:
        print(f"City '{city}' not found. Proceeding to next city.")
        pass
    
print("-------------------------------------------")
print(f"Successfully retrieved data for {len(successful_cities)} cities.")
print("-------------------------------------------")

-----------------------------------
Retrieving Data – Please be patient
-----------------------------------
Found data for city #1: busselton
Found data for city #2: hithadhoo
Found data for city #3: katsuura
Found data for city #4: jamestown
Found data for city #5: ushuaia
Found data for city #6: constitucion
City 'labutta' not found. Proceeding to next city.
Found data for city #7: punta arenas
Found data for city #8: sibolga
City 'illoqqortoormiut' not found. Proceeding to next city.
-----------------------------------
Successfully retrieved data for 8 cities.
-----------------------------------


In [53]:
# Create a dataframe
weather_df = pd.DataFrame({
    "City":successful_cities,
    "Temperature (°F)":temp,
    "Humidity (%)":humidity,
    "Cloudiness (%)":cloudiness,
    "Windspeed (mph)":windspeed
})

weather_df

# Export data into a csv


,City,Temperature (°F),Humidity (%),Cloudiness (%),Windspeed (mph)
0,busselton,55.36,97,20,7.36
1,hithadhoo,82.83,79,8,14.27
2,katsuura,35.91,35,96,1.01
3,jamestown,20.28,67,40,6.91
4,ushuaia,62.60,59,0,10.36
5,constitucion,67.51,67,27,8.70
6,punta arenas,69.39,40,0,14.97
7,sibolga,77.49,80,81,4.07
